#  Molecular Fingerprints

Molecules can be represented in many ways.  This tutorial introduces a type of representation called a "molecular fingerprint".  It is a very simple representation that often works well for small drug-like molecules.

## Colab

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Molecular_Fingerprints.ipynb)




In [1]:
!pip install --pre deepchem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.0/773.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 34.0 MB/s eta 0:00:00


We can now import the `deepchem` package to play with.

In [2]:
import deepchem as dc
import os
import pandas as pd
from deepchem.molnet.load_function.molnet_loader import TransformerGenerator, _MolnetLoader
from google.colab import drive
import traceback

from deepchem.data import Dataset
from typing import List, Optional, Tuple, Union

drive.mount('/content/drive')
dc.__version__

Mounted at /content/drive


'2.7.2.dev'

# What is a Fingerprint?

Deep learning models almost always take arrays of numbers as their inputs.  If we want to process molecules with them, we somehow need to represent each molecule as one or more arrays of numbers.

Many (but not all) types of models require their inputs to have a fixed size.  This can be a challenge for molecules, since different molecules have different numbers of atoms.  If we want to use these types of models, we somehow need to represent variable sized molecules with fixed sized arrays.

Fingerprints are designed to address these problems.  A fingerprint is a fixed length array, where different elements indicate the presence of different features in the molecule.  If two molecules have similar fingerprints, that indicates they contain many of the same features, and therefore will likely have similar chemistry.

DeepChem supports a particular type of fingerprint called an "Extended Connectivity Fingerprint", or "ECFP" for short.  They also are sometimes called "circular fingerprints".  The ECFP algorithm begins by classifying atoms based only on their direct properties and bonds.  Each unique pattern is a feature.  For example, "carbon atom bonded to two hydrogens and two heavy atoms" would be a feature, and a particular element of the fingerprint is set to 1 for any molecule that contains that feature.  It then iteratively identifies new features by looking at larger circular neighborhoods.  One specific feature bonded to two other specific features becomes a higher level feature, and the corresponding element is set for any molecule that contains it.  This continues for a fixed number of iterations, most often two.

Let's take a look at a dataset that has been featurized with ECFP.

In [3]:
"""
Delaney dataset loader.
"""

DELANEY_URL = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/delaney-processed.csv"
DELANEY_TASKS = ['measured log solubility in mols per litre']


class My(_MolnetLoader):

    def create_dataset(self) -> Dataset:
        dataset_file = os.path.join(self.data_dir, "delaney-processed.csv")
        # if not os.path.exists(dataset_file):
        #     dc.utils.data_utils.download_url(url=DELANEY_URL,
        #                                      dest_dir=self.data_dir)
        loader = dc.data.CSVLoader(tasks=self.tasks,
                                   feature_field="smiles",
                                   featurizer=self.featurizer)
        return loader.create_dataset(dataset_file, shard_size=8192)


def load_delaney(
    featurizer: Union[dc.feat.Featurizer, str] = 'ECFP',
    splitter: Union[dc.splits.Splitter, str, None] = 'scaffold',
    transformers: List[Union[TransformerGenerator, str]] = ['normalization'],
    reload: bool = True,
    data_dir: Optional[str] = "/content/drive/MyDrive",
    save_dir: Optional[str] = "/content/drive/MyDrive",
    **kwargs
) -> Tuple[List[str], Tuple[Dataset, ...], List[dc.trans.Transformer]]:
    """Load Delaney dataset
    The Delaney (ESOL) dataset a regression dataset containing structures and
    water solubility data for 1128 compounds. The dataset is widely used to
    validate machine learning models on estimating solubility directly from
    molecular structures (as encoded in SMILES strings).
    Scaffold splitting is recommended for this dataset.
    The raw data csv file contains columns below:
    - "Compound ID" - Name of the compound
    - "smiles" - SMILES representation of the molecular structure
    - "measured log solubility in mols per litre" - Log-scale water solubility
        of the compound, used as label
    Parameters
    ----------
    featurizer: Featurizer or str
        the featurizer to use for processing the data.  Alternatively you can pass
        one of the names from dc.molnet.featurizers as a shortcut.
    splitter: Splitter or str
        the splitter to use for splitting the data into training, validation, and
        test sets.  Alternatively you can pass one of the names from
        dc.molnet.splitters as a shortcut.  If this is None, all the data
        will be included in a single dataset.
    transformers: list of TransformerGenerators or strings
        the Transformers to apply to the data.  Each one is specified by a
        TransformerGenerator or, as a shortcut, one of the names from
        dc.molnet.transformers.
    reload: bool
        if True, the first call for a particular featurizer and splitter will cache
        the datasets to disk, and subsequent calls will reload the cached datasets.
    data_dir: str
        a directory to save the raw data in
    save_dir: str
        a directory to save the dataset in
    References
    ----------
    .. [1] Delaney, John S. "ESOL: estimating aqueous solubility directly from
        molecular structure." Journal of chemical information and computer
        sciences 44.3 (2004): 1000-1005.
    """
    loader = My(featurizer, splitter, transformers, DELANEY_TASKS,
                            data_dir, save_dir, **kwargs)
    return loader.load_dataset('delaney', reload)

In [21]:
"""
Tox21 dataset loader.
"""
# import os
# import deepchem as dc
# from deepchem.molnet.load_function.molnet_loader import TransformerGenerator, _MolnetLoader
# from deepchem.data import Dataset
# from typing import List, Optional, Tuple, Union

# TOX21_URL = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/tox21.csv.gz"
TOX21_TASKS = [
    'NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD',
    'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53'
]
DELANEY_TASKS = ['mouse_intraperitoneal_LD50',
       'mammal (species unspecified)_intraperitoneal_LD50',
       'guinea pig_intraperitoneal_LD50', 'rat_intraperitoneal_LD50',
       'rabbit_intraperitoneal_LD50', 'mouse_intraperitoneal_LDLo',
       'rat_intraperitoneal_LDLo', 'mouse_intravenous_LD50',
       'guinea pig_intravenous_LD50', 'rat_intravenous_LD50',
       'rabbit_intravenous_LD50', 'dog_intravenous_LD50',
       'cat_intravenous_LD50', 'mouse_intravenous_LDLo',
       'guinea pig_intravenous_LDLo', 'rat_intravenous_LDLo',
       'rabbit_intravenous_LDLo', 'dog_intravenous_LDLo',
       'cat_intravenous_LDLo', 'mouse_oral_LD50',
       'mammal (species unspecified)_oral_LD50', 'guinea pig_oral_LD50',
       'rat_oral_LD50', 'rabbit_oral_LD50', 'dog_oral_LD50', 'cat_oral_LD50',
       'bird - wild_oral_LD50', 'quail_oral_LD50', 'duck_oral_LD50',
       'chicken_oral_LD50', 'mouse_oral_LDLo', 'rat_oral_LDLo',
       'rabbit_oral_LDLo', 'dog_oral_LDLo', 'cat_oral_LDLo', 'man_oral_TDLo',
       'women_oral_TDLo', 'human_oral_TDLo', 'mouse_unreported_LD50',
       'mammal (species unspecified)_unreported_LD50', 'rat_unreported_LD50',
       'mouse_skin_LD50', 'guinea pig_skin_LD50', 'rat_skin_LD50',
       'rabbit_skin_LD50', 'rabbit_skin_LDLo', 'mouse_subcutaneous_LD50',
       'mammal (species unspecified)_subcutaneous_LD50',
       'guinea pig_subcutaneous_LD50', 'rat_subcutaneous_LD50',
       'rabbit_subcutaneous_LD50', 'mouse_subcutaneous_LDLo',
       'guinea pig_subcutaneous_LDLo', 'rat_subcutaneous_LDLo',
       'rabbit_subcutaneous_LDLo', 'frog_subcutaneous_LDLo',
       'mouse_intramuscular_LD50', 'rat_intramuscular_LD50',
       'mouse_parenteral_LD50']


class MyLoader(_MolnetLoader):

    def create_dataset(self) -> Dataset:
        # self.transformers = ['normalization']
        dataset_file = os.path.join(self.data_dir, "dataset.csv")
        # dataset_file = os.path.join(self.data_dir, "3.csv")
        # dataset_file = os.path.join(self.data_dir, "dataset.csv")
        # if not os.path.exists(dataset_file):
        #     dc.utils.data_utils.download_url(url=TOX21_URL,
        #                                      dest_dir=self.data_dir)
        loader = dc.data.CSVLoader(tasks=self.tasks,
                                   feature_field="SMILES",
                                   featurizer=self.featurizer)
        return loader.create_dataset(dataset_file, shard_size=8192)


def load_tox21(
    featurizer: Union[dc.feat.Featurizer, str] = 'ECFP',
    splitter: Union[dc.splits.Splitter, str, None] = 'scaffold',
    transformers: List[Union[TransformerGenerator, str]] = ['balancing'],
    reload: bool = False,
    data_dir: Optional[str] = "/content/drive/MyDrive",
    save_dir: Optional[str] = "/content/drive/MyDrive",
    **kwargs
) -> Tuple[List[str], Tuple[Dataset, ...], List[dc.trans.Transformer]]:
    """Load Tox21 dataset
    The "Toxicology in the 21st Century" (Tox21) initiative created a public
    database measuring toxicity of compounds, which has been used in the 2014
    Tox21 Data Challenge. This dataset contains qualitative toxicity measurements
    for 8k compounds on 12 different targets, including nuclear receptors and
    stress response pathways.
    Random splitting is recommended for this dataset.
    The raw data csv file contains columns below:
    - "smiles" - SMILES representation of the molecular structure
    - "NR-XXX" - Nuclear receptor signaling bioassays results
    - "SR-XXX" - Stress response bioassays results
    please refer to https://tripod.nih.gov/tox21/challenge/data.jsp for details.
    Parameters
    ----------
    featurizer: Featurizer or str
        the featurizer to use for processing the data.  Alternatively you can pass
        one of the names from dc.molnet.featurizers as a shortcut.
    splitter: Splitter or str
        the splitter to use for splitting the data into training, validation, and
        test sets.  Alternatively you can pass one of the names from
        dc.molnet.splitters as a shortcut.  If this is None, all the data
        will be included in a single dataset.
    transformers: list of TransformerGenerators or strings
        the Transformers to apply to the data.  Each one is specified by a
        TransformerGenerator or, as a shortcut, one of the names from
        dc.molnet.transformers.
    reload: bool
        if True, the first call for a particular featurizer and splitter will cache
        the datasets to disk, and subsequent calls will reload the cached datasets.
    data_dir: str
        a directory to save the raw data in
    save_dir: str
        a directory to save the dataset in
    References
    ----------
    .. [1] Tox21 Challenge. https://tripod.nih.gov/tox21/challenge/
    """
    loader = MyLoader(featurizer, splitter, transformers, DELANEY_TASKS,
                          data_dir, save_dir, **kwargs)
    return loader.load_dataset('tox21', reload)

In [22]:
# featurizer_name = str(self.featurizer)
# splitter_name = 'None' if self.splitter is None else str(self.splitter)
# save_folder = os.path.join(self.save_dir, name + "-featurized", featurizer_name, splitter_name)
# if len(self.transformers) > 0:
#   transformer_name = '_'.join(
#   t.get_directory_name() for t in self.transformers)
#   save_folder = os.path.join(save_folder, transformer_name)

#         # Create the dataset

#         logger.info("About to featurize %s dataset." % name)
#         dataset = self.create_dataset()

#         # Split and transform the dataset.

#         if self.splitter is None:
#             transformer_dataset: Dataset = dataset
#         else:
#             logger.info("About to split dataset with {} splitter.".format(
#                 self.splitter.__class__.__name__))
#             train, valid, test = self.splitter.train_valid_test_split(dataset)
#             transformer_dataset = train
#         transformers = [
#             t.create_transformer(transformer_dataset) for t in self.transformers
#         ]
#         logger.info("About to transform data.")
#         if self.splitter is None:
#             for transformer in transformers:
#                 dataset = transformer.transform(dataset)
#             if reload and isinstance(dataset, DiskDataset):
#                 dataset.move(save_folder)
#                 dc.utils.data_utils.save_transformers(save_folder, transformers)
#             return self.tasks, (dataset,), transformers

#         for transformer in transformers:
#             train = transformer.transform(train)
#             valid = transformer.transform(valid)
#             test = transformer.transform(test)
#         if reload and isinstance(train, DiskDataset) and isinstance(
#                 valid, DiskDataset) and isinstance(test, DiskDataset):
#             dc.utils.data_utils.save_dataset_to_disk(save_folder, train, valid,
#                                                      test, transformers)
#         return self.tasks, (train, valid, test), transformers

In [23]:
# TOX21_TASKS=['measured log solubility in mols per litre']
# class MyDataLoader(_MolnetLoader):

#     def create_dataset(self):
#         dataset_file = "/content/drive/MyDrive/delaney-processed.csv"
#         # if not os.path.exists(dataset_file):
#         #     dc.utils.data_utils.download_url(url=TOX21_URL,
#         #                                      dest_dir=self.data_dir)
#         loader = dc.data.CSVLoader(tasks=self.tasks,
#                                    feature_field="smiles",
#                                    featurizer=self.featurizer)
#         return loader.create_dataset(dataset_file, shard_size=8192)

# loader = MyDataLoader('ECFP', 'scaffold', ['balancing'], TOX21_TASKS, "", "save")

In [24]:
tasks, datasets, transformers = load_tox21(featurizer='ECFP')
# print(pd.read_csv("/content/drive/MyDrive/delaney-processed.csv").head())
# tasks, datasets, transformers = loader.load_dataset('delaney', False)
# train_dataset, valid_dataset, test_dataset = datasets
# print(train_dataset)
# train_dataset.to_dataframe().head(10)


The feature array `X` has shape (6264, 1024).  That means there are 6264 samples in the training set.  Each one is represented by a fingerprint of length 1024.  Also notice that the label array `y` has shape (6264, 12): this is a multitask dataset.  Tox21 contains information about the toxicity of molecules.  12 different assays were used to look for signs of toxicity.  The dataset records the results of all 12 assays, each as a different task.

Let's also take a look at the weights array.

In [ ]:
!head -n 4000 /usr/local/lib/python3.9/dist-packages/pandas/core/indexes/base.py | tail -n 500

        elif not self._should_compare(other):
            # We can infer that the intersection is empty.
            if isinstance(self, ABCMultiIndex):
                return self[:0].rename(result_name)
            return Index([], name=result_name)

        elif not is_dtype_equal(self.dtype, other.dtype):
            dtype = self._find_common_type_compat(other)
            this = self.astype(dtype, copy=False)
            other = other.astype(dtype, copy=False)
            return this.intersection(other, sort=sort)

        result = self._intersection(other, sort=sort)
        return self._wrap_intersection_result(other, result)

    def _intersection(self, other: Index, sort=False):
        """
        intersection specialized to the case with matching dtypes.
        """
        if (
            self.is_monotonic_increasing
            and other.is_monotonic_increasing
            and self._can_use_libjoin
        ):
            try:
                result = self._inner_indexer(o

In [ ]:
train_dataset.w

array([[451.        ],
       [225.5       ],
       [902.        ],
       [902.        ],
       [902.        ],
       [902.        ],
       [902.        ],
       [902.        ],
       [902.        ],
       [902.        ],
       [300.66666667],
       [180.4       ],
       [902.        ],
       [902.        ],
       [451.        ],
       [451.        ],
       [451.        ],
       [451.        ],
       [902.        ],
       [300.66666667],
       [225.5       ],
       [902.        ],
       [225.5       ],
       [902.        ],
       [902.        ],
       [300.66666667],
       [451.        ],
       [180.4       ],
       [180.4       ],
       [451.        ],
       [451.        ],
       [225.5       ],
       [225.5       ],
       [300.66666667],
       [300.66666667],
       [902.        ],
       [300.66666667],
       [300.66666667],
       [300.66666667],
       [225.5       ],
       [150.33333333],
       [902.        ],
       [902.        ],
       [300

Notice that some elements are 0.  The weights are being used to indicate missing data.  Not all assays were actually performed on every molecule.  Setting the weight for a sample or sample/task pair to 0 causes it to be ignored during fitting and evaluation.  It will have no effect on the loss function or other metrics.

Most of the other weights are close to 1, but not exactly 1.  This is done to balance the overall weight of positive and negative samples on each task.  When training the model, we want each of the 12 tasks to contribute equally, and on each task we want to put equal weight on positive and negative samples.  Otherwise, the model might just learn that most of the training samples are non-toxic, and therefore become biased toward identifying other molecules as non-toxic.

# Training a Model on Fingerprints

Let's train a model.  In earlier tutorials we use `GraphConvModel`, which is a fairly complicated architecture that takes a complex set of inputs.  Because fingerprints are so simple, just a single fixed length array, we can use a much simpler type of model.

In [ ]:
model = dc.models.MultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[1000])

`MultitaskClassifier` is a simple stack of fully connected layers.  In this example we tell it to use a single hidden layer of width 1000.  We also tell it that each input will have 1024 features, and that it should produce predictions for 12 different tasks.

Why not train a separate model for each task?  We could do that, but it turns out that training a single model for multiple tasks often works better.  We will see an example of that in a later tutorial.

Let's train and evaluate the model.

In [ ]:
import numpy as np

model.fit(train_dataset, nb_epoch=10)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train_dataset, [metric], transformers))
print('test set score:', model.evaluate(test_dataset, [metric], transformers))

ValueError: ignored

Not bad performance for such a simple model and featurization.  More sophisticated models do slightly better on this dataset, but not enormously better.

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!

## Citing This Tutorial
If you found this tutorial useful please consider citing it using the provided BibTeX. 

In [ ]:
@manual{Intro4, 
 title={Molecular Fingerprints}, 
 organization={DeepChem},
 author={Ramsundar, Bharath}, 
 howpublished = {\url{https://github.com/deepchem/deepchem/blob/master/examples/tutorials/Molecular_Fingerprints.ipynb}}, 
 year={2021}, 
} 

SyntaxError: ignored